In [1]:
import dawdreamer as daw
import numpy 
from scipy.io import wavfile
from pathlib import Path
import librosa
import librosa.display
import matplotlib.pyplot as plt

Load DAWDreamer engine

In [2]:
engine = daw.RenderEngine(44100, 512)

In [3]:
engine.set_bpm(120.)

In [4]:
synth = engine.make_plugin_processor("synth", "C:\\Program Files\\Common Files\\VST3\\TAL-NoiseMaker.vst3\\Contents\\x86_64-win\\TAL-NoiseMaker.vst3")

Check initial parameters of synth

In [6]:
synth.get_parameters_description()

[{'index': 0,
  'name': '-',
  'numSteps': 2147483647,
  'isBoolean': False,
  'isDiscrete': False,
  'label': '',
  'category': 'genericParameter',
  'text': '0.5000000',
  'currentValText': '0.5000000',
  'isMetaParameter': False,
  'isAutomatable': True,
  'defaultValue': 0.5,
  'defaultValueText': '0.5000000',
  'min': '0.0000000',
  'max': '1.0000000',
  'valueStrings': []},
 {'index': 1,
  'name': 'Master Volume',
  'numSteps': 2147483647,
  'isBoolean': False,
  'isDiscrete': False,
  'label': '',
  'category': 'genericParameter',
  'text': '0.4080000',
  'currentValText': '0.4080000',
  'isMetaParameter': False,
  'isAutomatable': True,
  'defaultValue': 0.4999000132083893,
  'defaultValueText': '0.4999000',
  'min': '0.0000000',
  'max': '1.0000000',
  'valueStrings': []},
 {'index': 2,
  'name': 'Filter Type',
  'numSteps': 2147483647,
  'isBoolean': False,
  'isDiscrete': False,
  'label': '',
  'category': 'genericParameter',
  'text': '0.0000000',
  'currentValText': '0.00

In [7]:
def generate_preset(synth):
    # generate random float between 0 and 1, 88 times
    params = [numpy.random.rand() for _ in range(88)]
    print(params)
    # Set the parameters
    for i, param in enumerate(params):
        synth.set_parameter(i, param)
    return params
    

In [8]:
def export_sample(synth, preset_name):
    # (MIDI note, velocity, start sec, duration sec)
    synth.add_midi_note(60, 100, 0.0, 5.)
    engine.load_graph([
        (synth, [])
        ])
    engine.render(2.)
    audio = engine.get_audio()

    directory = Path('./samples/')
    directory.mkdir(parents=True, exist_ok=True)
    
    wavfile.write(directory / (preset_name + '.wav'), 44100, audio.transpose())

In [9]:
def export_preset(synth, num_to_generate=1):
    for i in range(num_to_generate):
        # Generate unique preset string
        preset_name = "preset_" + str(i)
        params = generate_preset(synth)

        # Save 'preset_name', and all params to one row in csv
        with open(f'presets.csv', 'a') as f:
            f.write(f'{preset_name},')
            for param in params:
                f.write(f'{param},')
            f.write('\n')
        
        # export sound
        export_sample(synth, preset_name)

In [10]:
export_preset(synth, 300)

[0.18498781805382536, 0.1177438641978178, 0.7848788307713526, 0.4729105500381877, 0.974751785852127, 0.9132288274488968, 0.9440231598065427, 0.698077824537121, 0.0972201116295548, 0.7258294443791756, 0.3048729180606947, 0.8766737463468377, 0.7112711430218321, 0.16840049666195567, 0.8654807963864735, 0.9441803979254998, 0.3257072844469895, 0.6198810893569274, 0.7821099383230118, 0.3665516696223826, 0.03937833850162764, 0.5385071155685067, 0.7142973773544917, 0.3685514084453567, 0.831400623270487, 0.6819470890930268, 0.2553355841681504, 0.15977813311551348, 0.5527379585606854, 0.2893568978789901, 0.21404088936832144, 0.054385715492973885, 0.3037694753931497, 0.3822754677850452, 0.5108466510707206, 0.3300075301873707, 0.2187900240620635, 0.9552117039863298, 0.784772886724517, 0.10288695334580789, 0.20354389469059264, 0.2886526522636189, 0.7422841203795358, 0.49189786038847794, 0.9873095694719436, 0.8611975627015013, 0.18741387852273006, 0.16748510486769108, 0.5665839330994671, 0.198278644

In [79]:
def load_preset(synth, preset_name):
    with open(f'presets.csv', 'r') as f:
        for line in f:
            if line.startswith(preset_name):
                params = line.split(',')[1:]
                for i, param in enumerate(params):
                    # if param is new line, skip
                    if param == '\n':
                        continue
                    synth.set_parameter(i, float(param))

try to load a preset, save it out, to see if it matches generated file

In [5]:
load_preset(synth, 'preset_3')
export_sample(synth, 'test')

NameError: name 'load_preset' is not defined